In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

In [0]:
############################################### Creation of neural network with Pytorch ##########################################################
'''

PyTorch is an open source machine learning library based on the Torch library, used in computer vision problems
and natural language processing developed by Facebook.

For this example, the base of california_housing_train will be used, which contains a set of characteristics (*) with average price per house:

(*):
longitude	
latitude	
housing_median_age	
total_rooms	
total_bedrooms	
population	
households	
median_income	
median_house_value	
ocean_proximity	


The idea is to predict prices for new data.
'''

In [0]:
#carregar base treino e teste
data_treino = pd.read_csv('/content/sample_data/california_housing_train.csv')
data_teste = pd.read_csv('/content/sample_data/california_housing_test.csv')


In [0]:
X = data_treino.drop('median_house_value',axis=1)
y = data_treino['median_house_value'] 

In [0]:
#struct neural network

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()

    #layer
    self.fc1 = nn.Linear(8,100) # 8 input neurons and 100 output
    self.fc2 = nn.Linear(100,100) # 100 input neurons and 100 output
    self.out = nn.Linear(100,1) # 100 input neurons and 1 output
  

  
  def forward(self,x):
    x = F.normalize(x) #Normalize vetor
    x = F.relu(self.fc1(x)) #Activation Function
    x = F.relu(self.fc2(x)) 

    y = self.out(x)

    return y


In [29]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=8, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [0]:
#Train
X_tensor = torch.from_numpy(data_treino.values[:,:-1]).float()
Y_tensor = torch.from_numpy(data_treino.values[:,-1]).float()

Y_tensor = torch.log(Y_tensor)

In [0]:
#Optimizer
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),lr=0.001)#parametros da rede


In [0]:
#loss function
criterion = nn.MSELoss()

In [36]:
for i in range(200):
  optimizer.zero_grad()
  prediction = net(X_tensor)
  loss = criterion(prediction, Y_tensor)
  loss.backward() #derive and return the gradients
  optimizer.step()

  prediction2 = torch.exp(net(X_tensor))
  loss = criterion(prediction2, torch.exp(Y_tensor))

  print("Loss Monitor:", torch.sqrt(loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([17000])) that is different to the input size (torch.Size([17000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5938, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5781, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5469, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5469, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.5000, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.4219, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.4219, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.4219, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.4062, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.4062, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.3438, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268.1406, grad_fn=<SqrtBackward>)
Monitor de loss: tensor(236268., grad_fn=<SqrtBackward>)
Monitor de loss: tensor(2362

In [0]:
#Check with test data
X_test_tensor = torch.from_numpy(data_teste.values[:,:-1]).float()
Y_test_tensor = torch.from_numpy(data_teste.values[:,-1]).float()

In [0]:
prediction_test = torch.exp(net(X_test_tensor))

In [0]:
prediction_test_np = prediction_test.detach().numpy()

In [40]:
#Check the error in relation to the forecast / reality
from sklearn.metrics import mean_squared_error
y_test = date_teste.values[:,-1]
np.sqrt(mean_squared_error(y_test,prediction_test_np))

144283.3843164716